# Marvel Character Alignment Predictor

### Project Goal
The goal of this project is to build a machine learning model that can predict whether a Marvel character is 'good' or 'bad' based on their physical statistics.

This notebook will walk through the entire process:
1.  **Data Loading and Exploration**
2.  **Data Cleaning and Preprocessing**
3.  **Model Training and Evaluation**

### Libraries Used
*   Pandas for data manipulation
*   Scikit-learn for building the predictive model

In [1]:
import pandas as pd

### 1. Data Loading and Initial Cleaning

First, we load the dataset from the CSV file. A key first step is to handle the custom missing values (`-` and `-99`) directly upon loading by using the `na_values` parameter. This is more efficient than cleaning them after the fact.

In [2]:
na_vals = ['-99', '-']
df = pd.read_csv('HeroesList.csv', na_values=na_vals)

In [3]:
df

,ID,Name,Alignment,Gender,EyeColor,Race,HairColor,Publisher,SkinColor,Height,Weight
0,0,A-Bomb,good,Male,yellow,Human,No Hair,Marvel Comics,NaN,203.0,441.0
1,1,Abe Sapien,good,Male,blue,Icthyo Sapien,No Hair,Dark Horse Comics,blue,191.0,65.0
2,2,Abin Sur,good,Male,blue,Ungaran,No Hair,DC Comics,red,185.0,90.0
3,3,Abomination,bad,Male,green,Human / Radiation,No Hair,Marvel Comics,NaN,203.0,441.0
4,4,Abraxas,bad,Male,blue,Cosmic Entity,Black,Marvel Comics,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
729,729,Yellowjacket II,good,Female,blue,Human,Strawberry Blond,Marvel Comics,NaN,165.0,52.0
730,730,Ymir,good,Male,white,Frost Giant,No Hair,Marvel Comics,white,304.8,NaN
731,731,Yoda,good,Male,brown,Yoda's species,White,George Lucas,green,66.0,17.0
732,732,Zatanna,good,Female,blue,Human,Black,DC Comics,NaN,170.0,57.0


In [4]:
df['Alignment'].value_counts()

Alignment
good       496
bad        207
neutral     24
Name: count, dtype: int64

### 2. Data Cleaning and Preprocessing

This is the most critical phase. The raw data is not ready for a machine learning model. We need to perform several steps to filter, clean, and transform the data into a usable format.

In [5]:
filt = df['Publisher'] == 'Marvel Comics'
df = df[df['Alignment'].notna()]

In [6]:
df.isna()

,ID,Name,Alignment,Gender,EyeColor,Race,HairColor,Publisher,SkinColor,Height,Weight
0,False,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
729,False,False,False,False,False,False,False,False,True,False,False
730,False,False,False,False,False,False,False,False,False,False,True
731,False,False,False,False,False,False,False,False,False,False,False
732,False,False,False,False,False,False,False,False,True,False,False


#### 2.1 - Filtering for the Marvel Universe

The original dataset contains characters from multiple publishers. Since our goal is to focus on Marvel, we will filter the DataFrame to keep only the rows where the 'Publisher' is 'Marvel Comics'.

**Important Note:** We add `.copy()` at the end of the filter. This ensures that our new `df` is an independent DataFrame, which prevents a common `SettingWithCopyWarning` in Pandas later on.

In [7]:
df = df[df['Publisher'] == 'Marvel Comics'].copy()

In [8]:
df

,ID,Name,Alignment,Gender,EyeColor,Race,HairColor,Publisher,SkinColor,Height,Weight
0,0,A-Bomb,good,Male,yellow,Human,No Hair,Marvel Comics,NaN,203.0,441.0
3,3,Abomination,bad,Male,green,Human / Radiation,No Hair,Marvel Comics,NaN,203.0,441.0
4,4,Abraxas,bad,Male,blue,Cosmic Entity,Black,Marvel Comics,NaN,NaN,NaN
5,5,Absorbing Man,bad,Male,blue,Human,No Hair,Marvel Comics,NaN,193.0,122.0
8,8,Agent 13,good,Female,blue,NaN,Blond,Marvel Comics,NaN,173.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...
726,726,X-Man,good,Male,blue,NaN,Brown,Marvel Comics,NaN,175.0,61.0
727,727,Yellow Claw,bad,Male,blue,NaN,No Hair,Marvel Comics,NaN,188.0,95.0
728,728,Yellowjacket,good,Male,blue,Human,Blond,Marvel Comics,NaN,183.0,83.0
729,729,Yellowjacket II,good,Female,blue,Human,Strawberry Blond,Marvel Comics,NaN,165.0,52.0


#### 2.2 - Handling Missing Numerical Data

The `Height` and `Weight` columns contain missing (`NaN`) values. A model cannot handle these, so we need to "impute" them, or fill them in with a reasonable guess. We will use the **median** value for each column. We choose the median over the mean because it is less sensitive to extreme outliers (like the Hulk's weight).

In [9]:
median_height = df["Height"].median()
df['Height'] = df['Height'].fillna(median_height)
df

,ID,Name,Alignment,Gender,EyeColor,Race,HairColor,Publisher,SkinColor,Height,Weight
0,0,A-Bomb,good,Male,yellow,Human,No Hair,Marvel Comics,NaN,203.0,441.0
3,3,Abomination,bad,Male,green,Human / Radiation,No Hair,Marvel Comics,NaN,203.0,441.0
4,4,Abraxas,bad,Male,blue,Cosmic Entity,Black,Marvel Comics,NaN,183.0,NaN
5,5,Absorbing Man,bad,Male,blue,Human,No Hair,Marvel Comics,NaN,193.0,122.0
8,8,Agent 13,good,Female,blue,NaN,Blond,Marvel Comics,NaN,173.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...
726,726,X-Man,good,Male,blue,NaN,Brown,Marvel Comics,NaN,175.0,61.0
727,727,Yellow Claw,bad,Male,blue,NaN,No Hair,Marvel Comics,NaN,188.0,95.0
728,728,Yellowjacket,good,Male,blue,Human,Blond,Marvel Comics,NaN,183.0,83.0
729,729,Yellowjacket II,good,Female,blue,Human,Strawberry Blond,Marvel Comics,NaN,165.0,52.0


In [10]:
median_weight = df['Weight'].median()
df['Weight'] = df['Weight'].fillna(median_weight)
df

,ID,Name,Alignment,Gender,EyeColor,Race,HairColor,Publisher,SkinColor,Height,Weight
0,0,A-Bomb,good,Male,yellow,Human,No Hair,Marvel Comics,NaN,203.0,441.0
3,3,Abomination,bad,Male,green,Human / Radiation,No Hair,Marvel Comics,NaN,203.0,441.0
4,4,Abraxas,bad,Male,blue,Cosmic Entity,Black,Marvel Comics,NaN,183.0,83.0
5,5,Absorbing Man,bad,Male,blue,Human,No Hair,Marvel Comics,NaN,193.0,122.0
8,8,Agent 13,good,Female,blue,NaN,Blond,Marvel Comics,NaN,173.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...
726,726,X-Man,good,Male,blue,NaN,Brown,Marvel Comics,NaN,175.0,61.0
727,727,Yellow Claw,bad,Male,blue,NaN,No Hair,Marvel Comics,NaN,188.0,95.0
728,728,Yellowjacket,good,Male,blue,Human,Blond,Marvel Comics,NaN,183.0,83.0
729,729,Yellowjacket II,good,Female,blue,Human,Strawberry Blond,Marvel Comics,NaN,165.0,52.0


In [11]:
filt = df['Name'] == 'Spider-Man'
df.loc[filt]

,ID,Name,Alignment,Gender,EyeColor,Race,HairColor,Publisher,SkinColor,Height,Weight
622,622,Spider-Man,good,Male,hazel,Human,Brown,Marvel Comics,NaN,178.0,74.0
623,623,Spider-Man,good,NaN,red,Human,Brown,Marvel Comics,NaN,178.0,77.0
624,624,Spider-Man,good,Male,brown,Human,Black,Marvel Comics,NaN,157.0,56.0


#### 2.3 - Cleaning and Encoding the Target Variable ('Alignment')

Our target variable, 'Alignment', needs to be cleaned before we can use it.
1.  First, we remove all rows where 'Alignment' is 'neutral' to simplify our problem into a binary classification (good vs. bad).
2.  Next, we convert the text values 'good' and 'bad' into numbers (1 and 0, respectively) so the machine learning model can understand them. We do this using a dictionary with the `.replace()` method.

In [12]:
notneutral = (df["Alignment"] != "neutral")
df = df[notneutral].copy()
df

,ID,Name,Alignment,Gender,EyeColor,Race,HairColor,Publisher,SkinColor,Height,Weight
0,0,A-Bomb,good,Male,yellow,Human,No Hair,Marvel Comics,NaN,203.0,441.0
3,3,Abomination,bad,Male,green,Human / Radiation,No Hair,Marvel Comics,NaN,203.0,441.0
4,4,Abraxas,bad,Male,blue,Cosmic Entity,Black,Marvel Comics,NaN,183.0,83.0
5,5,Absorbing Man,bad,Male,blue,Human,No Hair,Marvel Comics,NaN,193.0,122.0
8,8,Agent 13,good,Female,blue,NaN,Blond,Marvel Comics,NaN,173.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...
726,726,X-Man,good,Male,blue,NaN,Brown,Marvel Comics,NaN,175.0,61.0
727,727,Yellow Claw,bad,Male,blue,NaN,No Hair,Marvel Comics,NaN,188.0,95.0
728,728,Yellowjacket,good,Male,blue,Human,Blond,Marvel Comics,NaN,183.0,83.0
729,729,Yellowjacket II,good,Female,blue,Human,Strawberry Blond,Marvel Comics,NaN,165.0,52.0


In [13]:
replacement_map = {'good': 1, 'bad': 0}
df['Alignment'] = df['Alignment'].map(replacement_map)

### 3. Feature Selection and Final Preparation

Now that our data is clean, we need to formally separate it into our features and our target.
*   **X (Features):** These are the "clues" or inputs for our model. We will start with just `Height` and `Weight`.
*   **y (Target):** This is the "answer" we want our model to predict. This is the numeric `Alignment` column.

In [14]:
y = df["Alignment"]
X = df[['Height', 'Weight']]

### 4. Model Training and Evaluation

With our data prepared, we can now use the Scikit-learn library to build our predictive model.

#### 4.1 - Splitting Data into Training and Testing Sets

This is the most important step in machine learning. We split our data into two parts:
-   **Training Set (80%):** The data the model will learn from.
-   **Testing Set (20%):** The data the model has never seen, which we will use to get an honest evaluation of its performance.

We use `random_state=42` to ensure that we get the same "random" split every time we run the code, making our results reproducible.

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (299, 2)
X_test shape: (75, 2)


#### 4.2 - Training the Model

We will use a `LogisticRegression` model, which is a great baseline for binary classification problems. The `.fit()` method is the "learning" step, where the model analyzes the training data (`X_train` and `y_train`) to find the patterns.

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
model = LogisticRegression()

In [20]:
print(y_train.value_counts(dropna=False))

Alignment
1    210
0     89
Name: count, dtype: int64


In [21]:
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


#### 4.3 - Making Predictions and Evaluating Performance

Now we use our trained model to make predictions on the unseen test data (`X_test`). We then compare these predictions to the true answers (`y_test`) to calculate our model's accuracy.

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
predictions = model.predict(X_test)

In [24]:
accuracy = accuracy_score(y_test, predictions)

In [25]:
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 62.67%


## 4.4 - Feature Engineering: Enriching Data with Wikipedia

Our initial model achieved an accuracy of around 62%. This is a decent start, but we can likely improve it by providing the model with more meaningful features. The character's height and weight are not very descriptive of their alignment.

Our new strategy is to:
1.  Fetch each character's biography from an external source (Wikipedia).
2.  Create new numerical features based on the text in their biography (e.g., counting "hero" vs. "villain" keywords).
3.  Re-train our model with these new, richer features.

First, we need to install the `wikipedia` library to access its API.

In [26]:
%%capture
import sys
!{sys.executable} -m pip install wikipedia

### 4.4.1 - Scraping Character Biographies

Now, let's define a function that takes a character's name, searches for it on Wikipedia, and returns the first two sentences of their biography. We will wrap this in a `try-except` block to handle cases where a character might not have a Wikipedia page, preventing our code from crashing.

In [27]:
import warnings
from bs4 import GuessedAtParserWarning
import wikipedia
warnings.filterwarnings("ignore", category=GuessedAtParserWarning)
      
def get_wiki_bio(name):
    try:
        query = f"{name} (Marvel Comics)"
        return wikipedia.summary(query, sentences=2).lower()
    except:
        try:
            return wikipedia.summary(name, sentences=2).lower()
        except:
            return "no data"

print("Function defined! Ready to scrape.")

Function defined! Ready to scrape.


Now we will apply this function to the 'Name' column of our DataFrame to create a new `Wiki_Bio` column.

**Note:** This step can take a few minutes to run, as it is making a separate request to the Wikipedia API for each character in our dataset.

In [28]:
print("Starting scrape... please wait.")

df['Wiki_Bio'] = df['Name'].apply(get_wiki_bio)

print("Scraping Complete!")

df[['Name', 'Wiki_Bio']].head()

Starting scrape... please wait.
Scraping Complete!


,Name,Wiki_Bio
0,A-Bomb,"richard milhouse ""rick"" jones is a character a..."
3,Abomination,the abomination (emil blonsky) is a character ...
4,Abraxas,no data
5,Absorbing Man,"the absorbing man (carl ""crusher"" creel) is a ..."
8,Agent 13,the agent (rick mason) is a fictional characte...


In [29]:
df.to_csv('marvel_final_enriched.csv', index=False)
print("Data saved successfully to 'marvel_final_enriched.csv'.")

Data saved successfully to 'marvel_final_enriched.csv'.


### 4.4.2 - Creating 'Hero' and 'Villain' Scores from Text

With the biographies collected, we can now engineer our new features. The idea is to create a "Villain Score" and a "Hero Score" for each character.

1.  We'll define two lists of words: one with keywords associated with villains (`villain_keywords`) and one with keywords for heroes (`hero_keywords`).
2.  We'll then create a function that counts how many times words from these lists appear in a character's biography.

In [30]:
import pandas as pd
import numpy as np

villain_keywords = ['villain', 'evil', 'enemy', 'antagonist', 'criminal', 'murder', 'threat', 'hydra', 'sinister', 'conqueror']
hero_keywords = ['hero', 'avenger', 'shield', 'protagonist', 'defend', 'save', 'guardians', 'member', 'joined', 'team']

def count_words(text, word_list):
    count = 0
    if str(text) == "no data" or pd.isna(text): 
        return 0
    
    for word in word_list:
        if word in str(text):
            count += 1
    return count

print("Engineering features...")
df['Villain_Score'] = df['Wiki_Bio'].apply(lambda x: count_words(x, villain_keywords))
df['Hero_Score'] = df['Wiki_Bio'].apply(lambda x: count_words(x, hero_keywords))

print("Feature Engineering Complete.")
df[['Name', 'Villain_Score', 'Hero_Score']].head()

Engineering features...
Feature Engineering Complete.


,Name,Villain_Score,Hero_Score
0,A-Bomb,0,0
3,Abomination,0,0
4,Abraxas,0,0
5,Absorbing Man,0,0
8,Agent 13,0,0


In [31]:
print("Total Hero Points:", df['Hero_Score'].sum())
print("Total Villain Points:", df['Villain_Score'].sum())

Total Hero Points: 206
Total Villain Points: 107


## 4.5 - Re-training the Model with New Features

Now that we have our new `Villain_Score` and `Hero_Score` features, we can train a new Logistic Regression model to see if our performance has improved.

The process is the same as before:
1.  Select our features (this time including the new scores).
2.  Split the data into training and testing sets.
3.  Train the model on the training data.
4.  Make predictions on the test data and evaluate the accuracy.

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

features = ['Villain_Score', 'Hero_Score', 'Height', 'Weight']
X = df[features]
y = df['Alignment']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

new_model = LogisticRegression()
new_model.fit(X_train, y_train)

predictions = new_model.predict(X_test)
final_accuracy = accuracy_score(y_test, predictions)

print(f"\n==========================================")
print(f"OLD ACCURACY (Height/Weight only): 62.67%")
print(f"NEW ACCURACY (with Wiki Scores): {final_accuracy * 100:.2f}%")
print(f"==========================================")


OLD ACCURACY (Height/Weight only): 62.67%
NEW ACCURACY (with Wiki Scores): 77.33%


## 5. Conclusion: From Physical Stats to Narrative Context

This project successfully demonstrated that a machine learning model's performance is critically dependent on the quality and relevance of its features. We evolved our approach from a simplistic baseline to a more sophisticated, context-aware model.

### The Tale of Two Models: A Stark Comparison

The core of this project lies in the contrast between our initial and final models:

*   **Model 1: The Baseline (Physical Stats Only)**
    *   **Input:** Used only the character's `Height` and `Weight`.
    *   **Result:** Achieved a modest **62.67%** accuracy.
    *   **Insight:** This confirmed our initial hypothesis: a character's physical build is a poor and unreliable proxy for their morality.

*   **Model 2: The Enriched Model (Narrative Features)**
    *   **Input:** Enhanced the dataset by adding a `Villain_Score` and `Hero_Score` derived from each character's Wikipedia biography.
    *   **Result:** Reached a significantly higher **77.33%** accuracy.
    *   **Insight:** This proves that *who a character is* (their actions and story) is a far more powerful predictor of their alignment than *what they look like* (their physical stats).

### Quantifying the Impact

The leap in performance is not trivial. By engineering just two new features from an external data source, we achieved:
-   An **absolute accuracy increase of nearly 15 percentage points**.
-   A **23% relative improvement** over the baseline model.

This outcome is a powerful validation of our central thesis: in machine learning, **context is king**.

### Future Work

While the enriched model is a clear success, the following steps could push its performance even further:

*   **Advanced NLP:** Move beyond simple keyword counting to more sophisticated techniques like TF-IDF or word embeddings to capture richer semantic meaning from the biographies.
*   **Explore Other Models:** Test more complex algorithms like Random Forest or Gradient Boosting to see if they can uncover more intricate patterns in the data.
*   **Incorporate Categorical Data:** Properly encode and include features like `Race` and `Gender` to determine if they hold any predictive power.